In [53]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Load training CSV
train_data = pd.read_csv(r'/content/train.csv')

# Extract image URLs and labels
train_image_urls = train_data['image_link'].values
train_labels = train_data['entity_value'].values

# Encode labels if needed
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

num_classes = df['entity_value'].nunique()
print(f"Number of classes: {num_classes}")

train_labels_encoded = train_labels_encoded.astype(int)

# One-hot encoding for multi-class classification
y_train_one_hot = to_categorical(train_labels_encoded)

# Check unique values of encoded labels
print("Unique encoded labels:", np.unique(train_labels_encoded))

print(np.max(train_labels_encoded))

# Check shape of one-hot encoded labels
print("Shape of y_train_one_hot:", y_train_one_hot.shape)

# Print a few rows of one-hot encoded labels
print("First few rows of y_train_one_hot:\n", y_train_one_hot[:10])
print(to_categorical)

Number of classes: 16405
Unique encoded labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]
64
Shape of y_train_one_hot: (99, 65)
First few rows of y_train_one_hot:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [40]:
import numpy as np
import requests
from PIL import Image, ImageFile
from io import BytesIO
from tensorflow.keras.utils import Sequence

# This ensures truncated images can still be processed
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Function to load and preprocess images from URLs with error handling
def load_image_from_url(url, target_size=(224, 224)):
    try:
        response = requests.get(url, timeout=10)  # Set timeout for slow responses
        response.raise_for_status()  # Raise HTTPError for bad responses
        img = Image.open(BytesIO(response.content)).convert('RGB')  # Convert image to RGB
        img = img.resize(target_size)  # Resize to target size
        img_array = np.array(img) / 255.0  # Convert to numpy array and normalize
        return img_array
    except (requests.exceptions.RequestException, OSError) as e:
        print(f"Error loading image from {url}: {e}")
        # Return a placeholder (black image) if there's an error
        return np.zeros((*target_size, 3))

# Add a return statement to the __getitem__ method
class ImageDataGenerator(Sequence):
    def __init__(self, image_urls, labels, batch_size=32, target_size=(224, 224), shuffle=True):
        self.image_urls = image_urls
        self.labels = labels
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        # Return the number of batches per epoch
        return int(np.floor(len(self.image_urls) / self.batch_size))

    def __getitem__(self, index):
        # Generate one batch of data
        batch_urls = self.image_urls[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        # Load and preprocess the images
        images = np.array([load_image_from_url(url, self.target_size) for url in batch_urls])

        return images, np.array(batch_labels) # Added this return statement

    def on_epoch_end(self):
        # Optionally shuffle the data at the end of each epoch
        if self.shuffle:
            combined = list(zip(self.image_urls, self.labels))
            np.random.shuffle(combined)
            self.image_urls, self.labels = zip(*combined)




In [34]:
# Create training and validation generators
train_generator = ImageDataGenerator(train_image_urls, y_train_one_hot, batch_size=32)



In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Ensure input shape is clearly defined
input_shape = (224, 224, 3)  # Change to your specific shape if needed
num_classes = 10  # Number of output classes (change this as per your classification problem)

# Load the pretrained base model without the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Build your custom model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # This reduces the output to a 2D tensor
    layers.Dense(512, activation='relu'),  # Custom dense layer
    layers.Dense(num_classes, activation='softmax')  # Final output layer, adjust num_classes for your problem
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [54]:
# Create training and validation generators
train_generator = ImageDataGenerator(train_image_urls, y_train_one_hot, batch_size=32)

# Iterate through the generator and print the shape of the data
for images, labels in train_generator:
    print(images.shape, labels.shape)
    print(labels) # Print the labels to see the group_ids
    break  # Stop after the first batch

# Evaluate the model using the generator and specify steps
test_loss, test_acc = model.evaluate(train_generator, steps=len(train_generator))
print(f"Test Accuracy: {test_acc}")


(32, 224, 224, 3) (32, 65)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


AttributeError: 'NoneType' object has no attribute 'items'

In [56]:
# Save the model
model.save('image_entity_extraction_model.keras')


In [57]:
import pandas as pd

# Load the sample CSV file containing image URLs for prediction
sample_data = pd.read_csv('/content/sample_test.csv')

# Display first few rows of the CSV file
print(sample_data.head())


   index                                         image_link  group_id  \
0      0  https://m.media-amazon.com/images/I/41-NCxNuBx...    658003   
1      1  https://m.media-amazon.com/images/I/41-NCxNuBx...    658003   
2      2  https://m.media-amazon.com/images/I/417NJrPEk+...    939426   
3      3  https://m.media-amazon.com/images/I/417SThj+Sr...    276700   
4      4  https://m.media-amazon.com/images/I/417SThj+Sr...    276700   

                     entity_name  
0                          width  
1                          depth  
2  maximum_weight_recommendation  
3                        voltage  
4                        wattage  


In [ ]:
import numpy as np
import requests
from PIL import Image
from io import BytesIO

# Function to load and preprocess an image from a URL
def load_image_from_url(url, target_size=(224, 224)):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')
    img = img.resize(target_size)  # Resize the image to match the model's input size
    img_array = np.array(img) / 255.0  # Normalize the image
    return img_array


In [ ]:
# Extract image URLs from the CSV file
sample_image_urls = sample_data['image_link'].values

# Load and preprocess images from URLs
sample_images = []

for url in sample_image_urls:
    try:
        image = load_image_from_url(url)
        sample_images.append(image)
    except:
        print(f"Failed to load image from {url}")

# Convert the list of images to a numpy array for prediction
X_sample = np.array(sample_images)


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('image_entity_extraction_model.h5')


ValueError: Layer "dense" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_1126>, <KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_1127>]

In [ ]:
# Make predictions on the sample data
predictions = model.predict(X_sample)

# Print raw predictions (probabilities for each class)
print(predictions)

# If you used one-hot encoding for your labels during training, you may want to convert the predictions back to class labels:
predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)

# If you have a label encoder, you can reverse the encoding to get the original class names:
#predicted_labels = label_encoder.inverse_transform(predicted_classes)
#print(predicted_labels)


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 972ms/step
[[0.25087544 0.05539677 0.02483157 0.04061174 0.11820537 0.14449467
  0.11296121 0.05669275 0.13766807 0.05826243]
 [0.25087544 0.05539677 0.02483157 0.04061174 0.11820537 0.14449467
  0.11296121 0.05669275 0.13766807 0.05826243]
 [0.19249031 0.0279333  0.05394839 0.03621569 0.03793417 0.12718728
  0.0834114  0.16945708 0.1727202  0.09870222]
 [0.15074594 0.03931948 0.08146908 0.01914515 0.11027356 0.08293116
  0.13268824 0.13732494 0.09701772 0.14908464]
 [0.15074594 0.03931948 0.08146908 0.01914515 0.11027356 0.08293116
  0.13268824 0.13732494 0.09701772 0.14908464]
 [0.14366946 0.02671497 0.04596415 0.05515042 0.09946744 0.11335561
  0.1165256  0.12558624 0.15853186 0.11503433]
 [0.08154701 0.08713713 0.05184818 0.020394   0.09673997 0.14163707
  0.0961661  0.17193842 0.13205177 0.1205402 ]
 [0.08154701 0.08713713 0.05184818 0.020394   0.09673997 0.14163707
  0.0961661  0.17193842 0.13205177 0.1205402 ]
 [0.1531666  0.05156285 0.11377299 0.0674

In [ ]:
# Add the predictions to the DataFrame
#sample_data['predicted_labels'] = predicted_labels

# Save the CSV file with predictions
sample_data.to_csv('sample_data_with_predictions.csv', index=False)

# Display the first few rows of the updated DataFrame
print(sample_data.head())


   index                                         image_link  group_id  \
0      0  https://m.media-amazon.com/images/I/41-NCxNuBx...    658003   
1      1  https://m.media-amazon.com/images/I/41-NCxNuBx...    658003   
2      2  https://m.media-amazon.com/images/I/417NJrPEk+...    939426   
3      3  https://m.media-amazon.com/images/I/417SThj+Sr...    276700   
4      4  https://m.media-amazon.com/images/I/417SThj+Sr...    276700   

                     entity_name  
0                          width  
1                          depth  
2  maximum_weight_recommendation  
3                        voltage  
4                        wattage  
